### Imports and configuration

In [1]:
# setup variables

import os
import json
import boto3
import tqdm
from s2orc.config import S3_BUCKET_NAME, CURRENT_VERSION
from s2orc.api.s3_utils import download_from_s3

LOCAL_S2ORC_DIR = 's2orc-data'
s3_manifest_file = f'{CURRENT_VERSION}/manifest.json'
local_manifest_file = os.path.join(LOCAL_S2ORC_DIR, CURRENT_VERSION, 'manifest.json')


s3 = boto3.resource('s3')
bucket = s3.Bucket(S3_BUCKET_NAME)


### Try to filter psychology papers by using paper metadata tsv venue field

In [4]:
# overview of metadata
import pandas as pd
from collections import Counter

# read manifest file
with open(local_manifest_file, 'r') as f:
    manifest = json.load(f)

start = 0
span = 1


entries = sorted(manifest['files'], key=lambda x: x['seq_num'])[start:(start + span)]
counter = Counter()
for entry in tqdm.tqdm(entries[start:(start + span)]):
    local_metadata_path = os.path.join(LOCAL_S2ORC_DIR, entry['metadata'])
    metadata = pd.read_csv(local_metadata_path, sep='\t')
    venues = metadata['venue'].astype(str).str.strip()
    counter.update(venues)


100%|██████████| 1/1 [00:00<00:00,  5.34it/s]


In [5]:
import re
import numpy as np

m = re.findall(r'^.*psycho.*$', '\n'.join(counter.keys()), flags=re.IGNORECASE | re.MULTILINE)

names = np.array(list(counter.keys()))
counts = np.array(list(counter.values()))
mask = np.isin(names, m)

all_values = sum(counter.values())
all_values
psycho_venues_count = sum(counts[mask])
psycho_venues_count


8259

66

In [6]:
# from matplotlib import pyplot as plt
# import numpy as np
# from collections import Counter
# # metadata.dtypes
# sorted(Counter()[::-1]
# len(metadata)